In [27]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

# 生成数据集
true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

# 读取数据集
def load_array(data_arrays, batch_size, is_train=True):
    """构造一个Pytorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)


batch_size = 10
data_iter = load_array((features, labels), batch_size)

# nn是神经网络的缩写
from torch import nn

# Sequential将多个层串联在一起，可以不使用Sequential，但是以后几乎都是多层
# nn.Linear定义一个全连接层，她的每一个输入都通过矩阵-向量乘法得到它的每个输出
# 第一个参数是输入特征形状为2
# 第二个参数指定输出特征形状是单个标量，所以为1
net = nn.Sequential(nn.Linear(2, 1))
"""定义模型"""

# 初始化模型参数
# net[0]选择网络中的第一个图层,在这里也就是上面设置的 Linear
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)


loss = nn.MSELoss()
"""定义损失函数(平方L2范数)，返回样本损失的平均值"""


# 指定优化参数通过net.parameters()从我们的模型中获得，设置超参数字典
# 小批量随机梯度下降只需要设置lr值
trainer = torch.optim.SGD(net.parameters(), lr=0.03)
"""定义优化算法随机梯度下降"""


# 开始训练

num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X), y)
        trainer.zero_grad()
        l.backward()
        trainer.step()  # 更新参数
    l = loss(net(features), labels)
    print(f"epoch {epoch + 1}, loss {l:f}")

w = net[0].weight.data
print("w的估计误差：", true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print("b的估计误差：", true_b - b)



epoch 1, loss 0.000238
epoch 2, loss 0.000101
epoch 3, loss 0.000100
tensor([[ 1.9993, -3.4012]])
tensor([4.1994])
